In [1]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

import fs_modelling as fs
from prophet import Prophet

import plotly.express as px

**Data**

In [2]:
X_train = pd.read_csv('./data/X_train_data/X_train.csv', index_col = 'Unnamed: 0')
X_val = pd.read_csv('./data/X_val_data/X_val.csv', index_col = 'Unnamed: 0')

def import_all_csvs_as_vars(folder):
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            df_name = file.replace('.csv', '')
            df = pd.read_csv(os.path.join(folder, file), index_col = 'Unnamed: 0')
            globals()[df_name] = df
            # print(f"Loaded {df_name}")

# Import each CSV file as individual DataFrames
import_all_csvs_as_vars('data/y_train_data')
import_all_csvs_as_vars('data/y_val_data')

In [3]:
X_train.head()

,CHI Production Index,CHI Shipments Index,FRA Production Index,FRA Shipments Index,GER Production Index,GER Shipments Index,ITA Production Index,ITA Shipments Index,JAP Production Index,JAP Shipments Index,SWI Production Index,SWI Shipments Index,UK Production Index,UK Shipments Index,USA Production Index,USA Shipments Index,Europe Production Index,Europe Shipments Index,(W) Price of Base Metals,(W) Price of Energy,(W) Price of Metals & Minerals,(W) Price of Natural gas index,"(W) Price of Crude oil, average",(W) Price of Copper,USA EUR to LCU Conversion Rate,USA EE Producer Prices,UK EE Producer Prices,ITA EE Producer Prices,FRA EE Producer Prices,GER EE Producer Prices,CHI EE Producer Prices,USA Machinery & Equipment Index,(W) Machinery & Equipment Index,SWI Machinery & Equipment Index,UK Machinery & Equipment Index,ITA Machinery & Equipment Index,JAP Machinery & Equipment Index,FRA Machinery & Equipment Index,GER Machinery & Equipment Index,USA EE Production Index,(W) EE Production Index,SWI EE Production Index,UK EE Production Index,ITA EE Production Index,JAP EE Production Index,FRA EE Production Index,GER EE Production Index,CC_CHI,CC_FRA,CC_GER,CC_ITA,CC_JAP,CC_Europe,CC_SWI,CC_UK,CC_USA,BC_CHI,BC_FRA,BC_GER,BC_ITA,BC_JAP,BC_Europe,BC_SWI,BC_UK,BC_USA,stock_price,stock_price_change,stock_volume,Covid_Chi,Covid_Fra,Covid_Ger,Covid_Ita,Covid_Swi,Covid_UK,Covid_US,Clean_Chi,Clean_Fra,Clean_Ger,Clean_Ita,Clean_Swi,Clean_UK,Clean_US,Fossil_Chi,Fossil_Fra,Fossil_Ger,Fossil_Ita,Fossil_Swi,Fossil_UK,Fossil_US,Buildings_Ger,Year,Month,Semester,sin_Semester,cos_Semester,Quarter,sin_Quarter,cos_Quarter,sin_month,cos_month,WeekendDaysCount,TotalDaysInMonth,WeekendDaysPercentage,SundayCount,GerHolidayCount
2018-10-01,211.955755,211.955755,108.280608,122.451734,124.227879,137.741953,118.350514,122.456894,125.209957,124.793250,109.077781,104.594781,113.659322,112.318030,111.902540,127.808839,124.391967,130.989253,86.102586,100.222169,79.354986,89.570796,97.072264,82.545082,1.1484,110.700409,111.463669,105.297836,102.064743,109.119614,96.318329,111.422638,131.340118,106.816937,110.893450,129.389221,131.503786,114.720810,127.461136,112.853256,129.325775,112.970843,118.670791,93.001511,112.376774,97.849541,118.298233,102.8056,98.94864,101.5778,101.7588,100.38490,100.7815,100.31800,100.69820,101.5022,98.79642,101.0451,101.7476,100.9696,101.5259,100.9295,102.0386,102.0568,101.2177,91.70,-7.81,61460000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.49,37.32,24.59,7.42,4.27,12.65,113.47,368.15,3.84,26.43,14.74,0.30,10.65,213.52,11738.0,2018,10,2,-2.449294e-16,1.0,4,-2.449294e-16,1.000000e+00,-8.660254e-01,0.500000,8,31,25.806452,4,1
2018-11-01,220.519655,220.519655,99.636911,115.958210,127.404132,142.732193,107.719260,120.132032,122.624695,123.289888,112.241491,107.656238,115.088417,112.801011,110.880401,117.675874,123.070091,132.934130,83.841374,84.436807,77.500875,97.362468,78.841167,82.230939,1.1367,110.994026,111.668373,105.297836,102.064743,109.224838,95.370118,109.737129,129.976456,110.792831,112.119922,117.990173,127.880755,104.873100,132.987915,113.145294,128.236176,114.736013,120.467019,84.133400,111.907535,91.155960,117.163727,103.2386,98.71818,101.5438,101.6450,100.28500,100.6595,100.30690,100.43900,101.3958,98.45412,100.9598,101.5859,100.8509,101.5088,100.9174,101.8399,102.2486,101.0049,92.31,0.67,48250000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,167.99,39.31,22.61,8.20,4.98,13.11,119.73,406.89,5.48,29.52,14.79,0.31,11.74,204.12,11177.0,2018,11,2,-2.449294e-16,1.0,4,-2.449294e-16,1.000000e+00,-5.000000e-01,0.866025,8,30,26.666667,4,0
2018-12-01,241.846854,241.846854,94.690312,115.128469,120.518565,141.407661,88.783181,131.936099,122.991956,124.508413,115.405201,110.717696,101.556108,94.503733,106.257796,123.280134,113.858005,131.261348,82.733389,74.898746,76.071705,94.406578,68.268564,80.630361,1.1384,111.162231,112.794266,105.297836,102.166710,109.330063,94.994885,103.448280,124.202469,114.768725,99.446384,99.191734,128.125679,104.974617,137.363281,111

In [4]:
# holds all external data
#external_data = 

# holds all product data, including all columns related to dates
#product_data = 

In [5]:
# This will hold all future data about features that are not in the test data
future_data_predictions = pd.DataFrame()

original = X_train.copy()

In [6]:
columns_example = [
    "CHI Production Index", "CHI Shipments Index",
    "FRA Production Index", "FRA Shipments Index",
    "GER Production Index", "GER Shipments Index",
    "ITA Production Index", "ITA Shipments Index",
    "JAP Production Index", "JAP Shipments Index",
    "SWI Production Index", "SWI Shipments Index"
]

In [7]:
# Iterate through each column in 'gdk_features_to_predict'
for column in columns_example:
    # Isolate the current column into a new DataFrame 'df1'
    df1 = original[[column]].copy()
    # Reset the index of 'df1' and rename columns to fit Prophet's expected format
    data = (df1.reset_index() \
        .rename(columns={'index':'ds',
                        f'{column}':'y'}))
    # Initialize Prophet
    model = Prophet()
    # Fit the model to the data
    model.fit(data)
    
    # Create a DataFrame representing future dates to make predictions
    # 'periods=10' is the number of future points to predict and 'freq='MS' sets the frequency to monthly start
    future = model.make_future_dataframe(periods=10, freq='MS')

    # Forecast future dates
    forecast_Production_Index = model.predict(future)
    # Keep only the 'ds' (date) and 'yhat' (predicted value) columns --> simplicity
    forecast_Production_Index  = forecast_Production_Index[['ds', 'yhat']]
    # Set the date column as the index
    forecast_Production_Index= forecast_Production_Index.set_index('ds')
    # Add the forecasted values to the 'future_features' DataFrame
    future_data_predictions[column] = forecast_Production_Index['yhat'].values

20:27:49 - cmdstanpy - INFO - Chain [1] start processing
20:27:49 - cmdstanpy - INFO - Chain [1] done processing
20:27:50 - cmdstanpy - INFO - Chain [1] start processing
20:27:50 - cmdstanpy - INFO - Chain [1] done processing
20:27:50 - cmdstanpy - INFO - Chain [1] start processing
20:27:50 - cmdstanpy - INFO - Chain [1] done processing
20:27:50 - cmdstanpy - INFO - Chain [1] start processing
20:27:50 - cmdstanpy - INFO - Chain [1] done processing
20:27:50 - cmdstanpy - INFO - Chain [1] start processing
20:27:50 - cmdstanpy - INFO - Chain [1] done processing
20:27:50 - cmdstanpy - INFO - Chain [1] start processing
20:27:51 - cmdstanpy - INFO - Chain [1] done processing
20:27:51 - cmdstanpy - INFO - Chain [1] start processing
20:27:51 - cmdstanpy - INFO - Chain [1] done processing
20:27:51 - cmdstanpy - INFO - Chain [1] start processing
20:27:51 - cmdstanpy - INFO - Chain [1] done processing
20:27:51 - cmdstanpy - INFO - Chain [1] start processing
20:27:51 - cmdstanpy - INFO - Chain [1]

In [8]:
# Reset the index to use date as a regular column
forecast_Production_Index.reset_index(inplace=True)
# Add the date column to 'future_features' --> keep track of the forecast dates
future_data_predictions['ds'] = forecast_Production_Index['ds'].values
# Set date as the index of 'future_features' --> facilitate time series analysis
future_data_predictions.set_index('ds', inplace = True)

In [ ]:
future_data_predictions.tail()

,CHI Production Index,CHI Shipments Index,FRA Production Index,FRA Shipments Index,GER Production Index,GER Shipments Index,ITA Production Index,ITA Shipments Index,JAP Production Index,JAP Shipments Index,SWI Production Index,SWI Shipments Index
ds,,,,,,,,,,,,
2021-10-01,301.540607,301.540607,95.572187,112.110575,128.057386,147.787431,106.248298,119.491503,141.359838,143.008570,109.684411,105.690907
2021-11-01,315.777307,315.777307,88.765802,110.540669,133.762221,155.667993,99.275465,118.608952,143.094797,145.393562,112.113073,108.026768
2021-12-01,338.237259,338.237259,87.828027,114.174042,132.947095,160.600638,83.503887,132.086903,151.908768,155.323554,114.583723,110.351349
2022-01-01,254.833214,254.833214,81.090252,98.745987,114.907141,133.580997,89.928999,93.055194,138.295336,140.804910,111.819500,107.759796
2022-02-01,255.699789,255.699789,86.895187,106.969500,125.375934,145.754203,102.543705,108.358843,148.357399,151.456660,109.031735,105.178863


In [ ]:
# Assign data from May 1, 2022 until February 1, 2023 in a new DataFrame: 'future_features_predicted'
future_data = future_data_predictions.loc['2022-05-01':'2023-02-01']

# depois dar replace por future_data

In [13]:
fig = px.line(future_data_predictions, x=future_data_predictions.index, y=future_data_predictions.columns, title="Future Values",
              labels={"value": "Value", "TIME_PERIOD": "Date"})
fig.update_layout(xaxis_title="Date", yaxis_title="Value", 
                  legend_title="Countries")
fig.show()